# TASK 1

We have two populations Blue (privileged) and Red (unprivileged), with the Blue population being 9 times larger than the Red population.

Individuals from both populations are requesting to attend XAI training to improve competency in this important area. Number of places is limited. The administrators of the training have decided to give priority to enrolling individuals who may need this training in the future, although unfortunately it is difficult to predict who will benefit.

The decision rule adopted:
1. In the Red group, half of the people will find the skills useful in future and half will not. Administrators randomly allocate 50% of people to training.
2. in the Blue group, 80% of people will find the training useful in future and 20% will not, although of course it is not known who will find it useful. The administrators have built a predictive model based on user behaviour in predicting for whom it will be useful and whom will not. The model has the following performance:


| Red                   	| Will use XAI | Will not use XAI | Total       |
|---------------------------|--------------|------------------|-------------|
| Enrolled in training     	| 25           | 25               | 50          |
| not enrolled in training 	| 25           | 25               | 50          |
| Total                    	| 50           | 50               | 100         |

| Blue                     	| Will use XAI 	| Will not use XAI 	| Total 	|
|--------------------------	|--------------	|------------------	|-------	|
| Enrolled in training     	| 60           	| 5               	| 65    	|
| not enrolled in training 	| 20            | 15               	| 35    	|
| Total                    	| 80           	| 20               	| 100   	|


Task: Calculate the Demographic parity, equal opportunity and predictive rate parity coefficients for this decision rule.


```
Demographic parity:      0.7692307692307692
Equal opportunity:       0.6666666666666666
Predictive rate parity:  0.5416666666666666
```
Starred task: How can this decision rule be changed to improve its fairness?


# TASK 2

In [192]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv("compas/raw/compas-scores-two-years.csv")
df["race2"] = df["race"]
df["race2"].where(df["race2"] == "African-American", "Other", inplace=True)

px.histogram(df, x='race', color='is_recid', barmode='group', title="Recydywa w różnych rasach").show()
px.histogram(df, x='race2', color='is_recid', barmode='group', title="Recydywa w różnych rasach").show()

/tmp/ipykernel_30032/2863819341.py:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/home/jowita/Documents/Programowanie/Magisterka/ml_venv/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/home/jowita/Documents/Programowanie/Magisterka/ml_venv/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Jako atrybut chroniony wybrałam rasę, ponieważ jest to atrybut, który jest często omawiany w przypadku tego datasetu. W moim przypadku jest to podział na (Afroamerykanin vs inne rasy).

## TASK 2.2
Skorzystałam tutaj z modelu CatBoostClassifier bez ważenia danych.
```
Demographic parity:      1.8553440374729437
Equal opportunity:       1.3898487931518384
Predictive rate parity:  1.011655964991207
```

Według moich obliczeń (atrybutem chronionym jest rasa - Afroamerykanin vs inne rasy) model nie jest sprawiedliwy według metryk Demographic parity, Equal opportunity, ale jest uczciwy w przypadku Predictive rate parity. Powyższe obserwacje nie są zaskakujące, ponieważ jak widać na wykresie odsetek recydywistów w grupie Afroamerykanów jest znacząco wyższy niż w pozostałych rasach uwzględnionych razem - Afroamerykanie to jedyna rasa w której odsetek recydywistów jest większy niż 50%, co wpływa na Demographic parity. 


## TASK 2.3
Skorzystałam tutaj z modelu DecisionTreeClassifier (drzewo decyzyjne)
```
Demographic parity:      1.451827468344597
Equal opportunity:       1.0563455810302884
Predictive rate parity:  0.9826090418120779
```
Wyniki uzyskane przez drzewa decyzyjne znacząco różnią się od wyników Catboosta. Model drzew decyzyjnych jest bardzo sprawiedliwy według metryk Equal opportunity oraz Predictive rate parity, jednak nadal nie jest sprawiedliwy według Demographic parity. Również drzewa decyzyjne osiągają znacząco lepsze wyniki niż Catboost - prawdopodobnie dochodzi tu do overfittingu, co dla zbioru treningowego daje prawie idealny klasyfikator, co w przypadku zbiasowego zbioru treningowego (ponad 50% Afroamerykanów jest recydywistami, a mniej niż 40% u pozostałych ras razem jest recydywistami) daje nam w wyniku Demographic parity wynik 1.45 (dla perfekcyjnego klasyfikatora wynik powinien dążyć do około 1.35 - niesprawiedliwa wartość).
Ponieważ model drzew decyzyjnych ma bardzo wysokie accuracy, to dla Equal opportunity i Predictive rate parity wartości będą bliskie 1, ponieważ model ma bardzo wysoką skuteczność, co daje nam bardzo dużo wartości True Positive i bardzo mało True Negative i False Positive.

## TASK 2.4
Skorzystałam tutaj z modelu CatBoostClassifier z wagami, które są proporcjonalne do liczby osób w danej grupie - rasa Afroamerykanów vs inne rasy.
``` 
Demographic parity:      1.8198813148573916
Equal opportunity:       1.362815806707338
Predictive rate parity:  1.0113089279755945
```

W ten sposób udało się sprawić, że model jest bardziej sprawiedliwy niż wcześniej. Wyniki są nieco lepsze do wyników Catboosta bez wag - accuracy zostało zachowane, ale wartości metryk poszły trochę w lepszą stronę. Model jest sprawiedliwy według metryki Predictive rate parity, jednak dalej nie jest sprawiedliwy według Demographic parity i Equal opportunity. Mimo zbilansowania wyników za pomocą wag nie udało się zwiększyć accuracy modelu ani znacząco poprawić sprawiedliwości modelu. Może to się wiązać z tym, że wynik przewidywania nie zależy tylko od rasy, ale również od innych czynników w zbiorze treningowym np. wiek, płeć, liczba wcześniejszych przestępstw. Dodatkowo dodanie wag do równego rozkładu dla rasy Afroamerykan i pozostałych jest dosyć zbilansowane


## TASK 2.5
| Model                   	        | Performance | Demographic parity | Equal opportunity | Predictive rate parity   |
|-----------------------------------|-------------|--------------|-------------------|--------------------------|
| Catboost     	                    | 0.7526      | 1.8552       | 1.3898            | 1.0117                   |
| Drzewa decyzyjne 	                | 0.9272      | 1.4518       | 1.0563            | 0.9826                   |
| Catboost ze zbilansowanymi wagami | 0.7528      | 1.8199       | 1.3628            | 1.0113                   |

Wnioski:
Najlepszym modelem okazał się model drzew decyzyjnych, który osiągnął najwyższą wartość accuracy. Warto zauważyć, że dodatkowo drzewa decyzyjne są najbardziej sprawiedliwe pod względem Equal opportunity oraz Predictive rate parity, przy czym według metryk Equal opportunity i Predictive rate parity drzewa decyzyjne są również sprawiedliwe (pomiędzy 0.8 a 1.25) - model bardzo dobrze dopasował się w przypadku drzew do zbioru treningowego, więc te współczynniki będą bliskie 1, ponieważ model ma prawie skuteczność 100%, co daje nam bardzo dużo wartości True Positive i bardzo mało True Negative i False Positive. 
Różnica między Catboostem a Catboostem ze zbilansowanymi wagami jest niewielka, jednak model z wagami jest odrobinę bardziej sprawiedliwy, a w przypadku Predictive rate parity jest nawet identyczny z modelem drzew decyzyjnych.  
Wszystkie modele są sprawiedliwe według metryki Predictive rate parity, co oznacza, że model dobrze przewiduje, kto z danej rasy w przyszłości popełni recydywę - jeśli już model komuś przewidział recydywę to raczej jest ona pewna. Modele te niestety mają skłonność do przeszacowywania wyników False Negative, co oznacza, że osoby które popełnią recydywę w przyszłości nie zostaną zaklasyfikowane jako przyszli recydywiści. 


# APPENDIX

## TASK 1

We have two populations Blue (privileged) and Red (unprivileged), with the Blue population being 9 times larger than the Red population.

Individuals from both populations are requesting to attend XAI training to improve competency in this important area. Number of places is limited. The administrators of the training have decided to give priority to enrolling individuals who may need this training in the future, although unfortunately it is difficult to predict who will benefit.

The decision rule adopted:
1. In the Red group, half of the people will find the skills useful in future and half will not. Administrators randomly allocate 50% of people to training.
2. in the Blue group, 80% of people will find the training useful in future and 20% will not, although of course it is not known who will find it useful. The administrators have built a predictive model based on user behaviour in predicting for whom it will be useful and whom will not. The model has the following performance:


| Red                   	| Will use XAI | Will not use XAI | Total       |
|---------------------------|--------------|------------------|-------------|
| Enrolled in training     	| 25           | 25               | 50          |
| not enrolled in training 	| 25           | 25               | 50          |
| Total                    	| 50           | 50               | 100         |

| Blue                     	| Will use XAI 	| Will not use XAI 	| Total 	|
|--------------------------	|--------------	|------------------	|-------	|
| Enrolled in training     	| 60           	| 5               	| 65    	|
| not enrolled in training 	| 20            | 15               	| 35    	|
| Total                    	| 80           	| 20               	| 100   	|


Task: Calculate the Demographic parity, equal opportunity and predictive rate parity coefficients for this decision rule.

Starred task: How can this decision rule be changed to improve its fairness?



In [96]:
def calculate_demographic_parity(n_red, n_blue, n_red_enrolled, n_blue_enrolled):
    p_red = n_red_enrolled / n_red
    p_blue = n_blue_enrolled / n_blue
    print("Demographic parity: ")
    print("Result  = ", p_red / p_blue)
    
def calculate_equal_opportunity(n_red_useful, n_blue_useful, n_red_enrolled_useful, n_blue_enrolled_useful):
    p_red = n_red_enrolled_useful / n_red_useful
    p_blue = n_blue_enrolled_useful / n_blue_useful
    print("Equal opportunity: ")
    print("Result  = ", p_red / p_blue)
    
def calculate_predictive_rate_parity(n_red_enrolled, n_blue_enrolled, n_red_enrolled_useful, n_blue_enrolled_useful):
    p_red = n_red_enrolled_useful / n_red_enrolled
    p_blue = n_blue_enrolled_useful / n_blue_enrolled
    print("Predictive rate parity: ")
    print("Result  = ", p_red / p_blue)
    
calculate_demographic_parity(900, 100, 450, 65)
calculate_equal_opportunity(50, 80, 25, 60)
calculate_predictive_rate_parity(50, 65, 25, 60)
    

Demographic parity: 
Result  =  0.7692307692307692
Equal opportunity: 
Result  =  0.6666666666666666
Predictive rate parity: 
Result  =  0.5416666666666666


# TASK 2

In [190]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv("compas/raw/compas-scores-two-years.csv")

px.histogram(df, x='race', color='is_recid', barmode='group').show()
px.histogram(df, x='race2', color='is_recid', barmode='group').show()

3696 3518
race
African-American    51.233712
Caucasian           34.017189
Hispanic             8.830053
Other                5.225950
Asian                0.443582
Native American      0.249515
Name: count, dtype: float64


/tmp/ipykernel_30032/2314990525.py:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/home/jowita/Documents/Programowanie/Magisterka/ml_venv/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/home/jowita/Documents/Programowanie/Magisterka/ml_venv/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Wybrałam wspomniany na wykładzie dataset COMPAS, który zawiera dane dotyczące oceny ryzyka recydywy. Dataset zawiera informacje o 7214 osobach, które zostały aresztowane i ocenione w okresie dwóch lat. Dane zawierają:
- id: Unikalny identyfikator dla każdej osoby.
- name: Pełne imię i nazwisko osoby.
- first: Imię osoby.
- last: Nazwisko osoby.
- compas_screening_date: Data przeprowadzenia oceny COMPAS dla danej osoby.
- sex: Płeć osoby (Mężczyzna lub Kobieta).
- dob: Data urodzenia osoby.
- age: Wiek osoby w momencie oceny.
- age_cat: Kategoria wiekowa osoby (np. Mniej niż 25 lat, 25 - 45 lat, Więcej niż 45 lat).
- race: Rasa osoby (np. Afroamerykanin, Kaukaska, Inna).
- juv_fel_count: Liczba wyroków za przestępstwa popełnione jako nieletni.
- decile_score: Dziesiętny wynik COMPAS, oceniający ryzyko recydywy (od 1 do 10).
- juv_misd_count: Liczba wykroczeń popełnionych jako nieletni.
- juv_other_count: Liczba innych przestępstw popełnionych jako nieletni (nierejestrowanych jako wykroczenia lub przestępstwa).
- priors_count: Liczba wcześniejszych przestępstw.
- days_b_screening_arrest: Liczba dni pomiędzy oceną COMPAS a aresztowaniem.
- c_jail_in: Data i godzina, kiedy osoba trafiła do aresztu.
- c_jail_out: Data i godzina, kiedy osoba została wypuszczona z aresztu.
- c_case_number: Numer sprawy związanej z aktualnym zarzutem.
- c_offense_date: Data przestępstwa związanego z aktualnym zarzutem.
- c_arrest_date: Data aresztowania związanego z aktualnym zarzutem.
- c_days_from_compas: Liczba dni od oceny COMPAS do zarzutu.
- c_charge_degree: Stopień zarzutu (np. F dla przestępstwa, M dla wykroczenia).
- c_charge_desc: Opis aktualnego zarzutu.
- is_recid: Wskaźnik (0 lub 1), czy osoba ponownie popełni przestępstwo po początkowej ocenie.
- r_case_number: Numer sprawy związanej z recydywą (jeśli dotyczy).
- r_charge_degree: Stopień zarzutu związanego z recydywą (np. przestępstwo lub wykroczenie).
- r_days_from_arrest: Liczba dni pomiędzy aresztowaniem a recydywą (jeśli dotyczy).
- r_offense_date: Data przestępstwa związanego z recydywą.
- r_charge_desc: Opis zarzutu związanego z recydywą.
- r_jail_in: Data i godzina, kiedy osoba trafiła do aresztu związanego z recydywą.
- r_jail_out: Data i godzina, kiedy osoba została wypuszczona z aresztu związanego z recydywą.
- violent_recid: Wskaźnik (0 lub 1), czy osoba popełniła przestępstwo związane z recydywą o charakterze gwałtownym.
- is_violent_recid: Wskaźnik (0 lub 1), czy osoba popełniła przestępstwo związane z recydywą o charakterze gwałtownym.
- vr_case_number: Numer sprawy związanej z recydywą o charakterze gwałtownym (jeśli dotyczy).
- vr_charge_degree: Stopień zarzutu związanego z recydywą o charakterze gwałtownym.
- vr_offense_date: Data przestępstwa związanego z recydywą o charakterze gwałtownym.
- vr_charge_desc: Opis zarzutu związanego z recydywą o charakterze gwałtownym.
- type_of_assessment: Rodzaj oceny COMPAS (np. Ryzyko recydywy, Ryzyko przemocy).
- decile_score: Wynik dziesiętny z oceny COMPAS, oceniający ryzyko recydywy.
- score_text: Opis tekstowy ryzyka (np. Niskie, Średnie, Wysokie) na podstawie wyniku dziesiętnego.
- screening_date: Data przeprowadzenia oceny COMPAS dla danego typu oceny.
- v_type_of_assessment: Rodzaj oceny COMPAS związanej z ryzykiem recydywy o charakterze gwałtownym (np. Ryzyko przemocy).
- v_decile_score: Wynik dziesiętny z oceny COMPAS, oceniający ryzyko recydywy o charakterze gwałtownym.
- v_score_text: Opis tekstowy ryzyka recydywy o charakterze gwałtownym (np. Niskie, Średnie, Wysokie).
- v_screening_date: Data oceny COMPAS dotyczącej ryzyka recydywy o charakterze gwałtownym.
- in_custody: Data, kiedy osoba trafiła do aresztu (jeśli dotyczy).
- out_custody: Data, kiedy osoba została wypuszczona z aresztu (jeśli dotyczy).
- priors_count: Liczba wcześniejszych przestępstw (może być duplikatem wcześniejszej kolumny).
- start: Data rozpoczęcia wydarzenia związanego z recydywą (np. areszt lub przestępstwo).
- end: Data zakończenia wydarzenia związanego z recydywą.
- event: Opis wydarzenia (związanego z recydywą).
- two_year_recid: Wskaźnik (0 lub 1), czy osoba popełni recydywę w ciągu dwóch lat.

W swoim treningu wykorzystam tylko część danych z racji tego, że niektóre kolumny nie są istotne dla analizy - zawierają dublujące się informacje lub nieistotne dane.

## TASK 2.1

In [98]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    random_seed=42,
    cat_features=["race", "sex", "c_charge_degree"],
    logging_level='Silent',
    eval_metric='Accuracy'
)

def train_model(model, df, features, target):
    X = df[features]
    y = df[target]
    model.fit(X, y, eval_set=(X, y), plot=True) # eval_set = train set, bo zależy nam tylko na policzeniu accuracy


In [99]:
features = ["sex", "age", "race", "juv_fel_count", "decile_score", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree"]
PREDICTION = "is_predict_recid"
TRUE = "is_real_recid"
AFRICA = "African-American"

target = "is_recid" # czy osoba ponownie popełni przestępstwo po początkowej ocenie
train_model(model, df, features, target)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## TASK 2.2

Protected attributes is race (African-American vs other races)

In [100]:
# For race = African-American
df_analyse = pd.DataFrame()
df_analyse["race"] = df["race"]
df_analyse[PREDICTION] = model.predict(df[features])
df_analyse[TRUE] = df["is_recid"]

n_african_american = len(df_analyse[df_analyse["race"] == AFRICA])
n_other = len(df_analyse[df_analyse["race"] != AFRICA])
n_african_american_recid = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse["is_predict_recid"] == 1)])
n_other_recid = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse["is_predict_recid"] == 1)])
calculate_demographic_parity(n_african_american, n_other, n_african_american_recid, n_other_recid)

TP_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse["is_predict_recid"] == 1) & (df_analyse[TRUE] == 1)])
FN_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse["is_predict_recid"] == 0) & (df_analyse[TRUE] == 1)])
TP_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse["is_predict_recid"] == 1) & (df_analyse[TRUE] == 1)])
FN_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse["is_predict_recid"] == 0) & (df_analyse[TRUE] == 1)])
calculate_equal_opportunity(TP_african_american + FN_african_american, TP_other + FN_other, TP_african_american, TP_other)

FP_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse["is_predict_recid"] == 1) & (df_analyse[TRUE] == 0)])
FP_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse["is_predict_recid"] == 1) & (df_analyse[TRUE] == 0)])

calculate_predictive_rate_parity(TP_african_american + FP_african_american, TP_other + FP_other, TP_african_american, TP_other)



Demographic parity: 
Result  =  1.8553440374729437
Equal opportunity: 
Result  =  1.3898487931518384
Predictive rate parity: 
Result  =  1.011655964991207


## TASK 2.3

In [193]:
from sklearn.tree import DecisionTreeClassifier

# const features
tree_features = ["sex", "age", "juv_fel_count", "decile_score", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree"]

# data preparation
df_tree = df.copy()
df_tree["sex"] = (df_tree["sex"] == "Male")

all_races = df_tree["race"].unique()
for race in all_races:
    df_tree[race] = pd.Series(df_tree["race"] == race)

tree_features.extend(all_races)

df_tree["c_charge_degree"] = (df_tree["c_charge_degree"] == "F")
df_tree[TRUE] = df_tree["is_recid"]

# model prediction
tree = DecisionTreeClassifier()
tree.fit(df_tree[tree_features], df_tree[target])
print(tree.score(df_tree[tree_features], df_tree[target]))
df_tree[PREDICTION] = tree.predict(df_tree[tree_features])


# calculate metrics
P_african_american = (df_tree["race"] == AFRICA).sum()
P_other = (df_tree["race"] != AFRICA).sum()

RP_african_american = len(df_tree[(df_tree["race"] == AFRICA) & (df_tree[PREDICTION] == 1)])
RP_other = len(df_tree[(df_tree["race"] != AFRICA) & (df_tree[PREDICTION] == 1)])

calculate_demographic_parity(P_african_american, P_other, RP_african_american, RP_other)

TP_african_american = len(df_tree[(df_tree["race"] == AFRICA) & (df_tree[PREDICTION] == 1) & (df_tree[TRUE] == 1)])
FN_african_american = len(df_tree[(df_tree["race"] == AFRICA) & (df_tree[PREDICTION] == 0) & (df_tree[TRUE] == 1)])

TP_other = len(df_tree[(df_tree["race"] != AFRICA) & (df_tree[PREDICTION] == 1) & (df_tree[TRUE] == 1)])
FN_other = len(df_tree[(df_tree["race"] != AFRICA) & (df_tree[PREDICTION] == 0) & (df_tree[TRUE] == 1)])

calculate_equal_opportunity(TP_african_american + FN_african_american, TP_other + FN_other, TP_african_american, TP_other)

FP_african_american = len(df_tree[(df_tree["race"] == AFRICA) & (df_tree[PREDICTION] == 1) & (df_tree[TRUE] == 0)])
FP_other = len(df_tree[(df_tree["race"] != AFRICA) & (df_tree[PREDICTION] == 1) & (df_tree[TRUE] == 0)])

calculate_predictive_rate_parity(TP_african_american + FP_african_american, TP_other + FP_other, TP_african_american, TP_other)

0.927224840587746
Demographic parity: 
Result  =  1.451827468344597
Equal opportunity: 
Result  =  1.0563455810302884
Predictive rate parity: 
Result  =  0.9826090418120779


## TASK 2.4

In [207]:
model = CatBoostClassifier(
    random_seed=42,
    cat_features=["race", "sex", "c_charge_degree"],
    logging_level='Silent',
    eval_metric='Accuracy', 
)

def train_model(model, df, features, target):
    X = df[features]
    y = df[target]
    
    all_races = df["race2"].unique()
    df["weight"] = 0.0
    for race in all_races:
        value = (len(df) / len(all_races)) / (df["race2"] == race).sum()
        df["weight"].loc[df["race2"] == race] = value
        
    print(df["weight"].head())
        
    model.fit(X, y, eval_set=(X, y), plot=True, sample_weight=df["weight"]) # eval_set = train set, bo zależy nam tylko na policzeniu accuracy


In [208]:
train_model(model, df, features, target)

0    1.025298
1    0.975920
2    0.975920
3    0.975920
4    1.025298
Name: weight, dtype: float64


/tmp/ipykernel_30032/3733718453.py:16: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/tmp/ipykernel_30032/3733718453.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [209]:
# For race = African-American
df_analyse = pd.DataFrame()
df_analyse["race"] = df["race"]
df_analyse[PREDICTION] = model.predict(df[features])
df_analyse[TRUE] = df["is_recid"]

n_african_american = len(df_analyse[df_analyse["race"] == AFRICA])
n_other = len(df_analyse[df_analyse["race"] != AFRICA])
n_african_american_recid = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse[PREDICTION] == 1)])
n_other_recid = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse[PREDICTION] == 1)])
calculate_demographic_parity(n_african_american, n_other, n_african_american_recid, n_other_recid)

TP_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse[PREDICTION] == 1) & (df_analyse[TRUE] == 1)])
FN_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse[PREDICTION] == 0) & (df_analyse[TRUE] == 1)])
TP_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse[PREDICTION] == 1) & (df_analyse[TRUE] == 1)])
FN_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse[PREDICTION] == 0) & (df_analyse[TRUE] == 1)])
calculate_equal_opportunity(TP_african_american + FN_african_american, TP_other + FN_other, TP_african_american, TP_other)

FP_african_american = len(df_analyse[(df_analyse["race"] == AFRICA) & (df_analyse[PREDICTION] == 1) & (df_analyse[TRUE] == 0)])
FP_other = len(df_analyse[(df_analyse["race"] != AFRICA) & (df_analyse[PREDICTION] == 1) & (df_analyse[TRUE] == 0)])

calculate_predictive_rate_parity(TP_african_american + FP_african_american, TP_other + FP_other, TP_african_american, TP_other)

Demographic parity: 
Result  =  1.8198813148573916
Equal opportunity: 
Result  =  1.362815806707338
Predictive rate parity: 
Result  =  1.0113089279755945
